In [ ]:
%pip install -q numpy pandas matplotlib seaborn scikit-learn tensorflow h5py

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# pd.set_option('display.max_rows', None)
fp = "./features_combined.csv"
batch_pd = pd.read_csv(fp, index_col=False)
dataset = batch_pd.copy()
dataset.sort_values(by=['policy'], ascending=True, inplace=True)
dataset

In [ ]:
dataset.isna().sum()
dataset = dataset.dropna().drop(columns=['policy', 'barcode'])
dataset

## Data split

In [ ]:
normal_charge_dataset = dataset.iloc[0:29, :]
fast_charge_dataset = dataset.iloc[29:, :]

## Normal Charge Test-Train split

In [ ]:
normal_charge_train_ds = normal_charge_dataset.iloc[0::2, :]
normal_charge_test_ds = normal_charge_dataset.iloc[1::2, :]
sns.pairplot(normal_charge_train_ds[['cycle_life', 'QDiffLinVar']], diag_kind='kde')
sns.pairplot(normal_charge_test_ds[['cycle_life', 'QDiffLinVar']], diag_kind='kde')

In [ ]:
normal_charge_train_ds.describe().transpose()


In [ ]:
normalcharge_train_features = normal_charge_train_ds.copy()
normalcharge_test_features = normal_charge_test_ds.copy()

train_labels = normalcharge_train_features.pop('cycle_life')
test_labels = normalcharge_test_features.pop('cycle_life')


## Normalization Layer

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(normalcharge_train_features))
print(normalizer.mean.numpy())

# Linear Regress
## Layering

In [ ]:
QDiffLinVar = np.array(normalcharge_train_features['QDiffLinVar'])
QDiffLinVar_normalizer = layers.Normalization(input_shape=[1,], axis=None)
QDiffLinVar_normalizer.adapt(QDiffLinVar)

## Model Building

In [ ]:
variance_model_normal_charge = tf.keras.Sequential([
    QDiffLinVar_normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

variance_model_normal_charge.summary()
print("Number of weights after calling the model:", len(variance_model_normal_charge.weights))
print("weights:", len(variance_model_normal_charge.weights))
print("trainable_weights:", len(variance_model_normal_charge.trainable_weights))
print("non_trainable_weights:", len(variance_model_normal_charge.non_trainable_weights))


In [ ]:
variance_model_normal_charge.predict(QDiffLinVar[:10])

In [ ]:
variance_model_normal_charge.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
    )

In [ ]:
%%time
history = variance_model_normal_charge.fit(
    normalcharge_train_features['QDiffLinVar'],
    train_labels,
    epochs=1000,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
def plot_loss(history):
  plt.plot(np.sqrt(history.history['loss']), label='loss')
  plt.plot(np.sqrt(history.history['val_loss']), label='val_loss')
  # plt.ylim([200, 130])
  plt.ylim([50, 300])
  plt.xlabel('Epoch')
  plt.ylabel('Error [cycles]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
hist = pd.DataFrame(history.history)
hist = hist.pow(0.5) # Power 1/2 is the same as square root
hist['epoch'] = history.epoch
hist


In [ ]:
test_results = {}

test_results['normal_charge_variance_model'] = variance_model_normal_charge.evaluate(
    normalcharge_test_features['QDiffLinVar'],
    test_labels, verbose=1) #sqrt for mse

## Predict

In [ ]:
def plot_prediction(y_train, y_test):
  plt.axes(aspect='equal')
  plt.scatter(y_train, train_labels, label='Predictions (train)')
  plt.scatter(y_test, test_labels, label='Predictions (test)')
  lims = [0, 2000]
  plt.xlim(lims)
  plt.ylim(lims)
  plt.plot(lims, lims, 'k', )
  plt.xlabel('Predicted Cycle life')
  plt.ylabel('Actual Cycle life')
  plt.legend()

train_prediction = variance_model_normal_charge.predict(normal_charge_train_ds['QDiffLinVar'])
test_prediction = variance_model_normal_charge.predict(normal_charge_test_ds['QDiffLinVar'])
plot_prediction(train_prediction, test_prediction)
print(train_prediction.size, test_prediction.size)

In [ ]:
print("weights:", len(variance_model_normal_charge.weights))
print("trainable_weights:", len(variance_model_normal_charge.trainable_weights))
print("non_trainable_weights:", len(variance_model_normal_charge.non_trainable_weights))


In [ ]:
test_results['normal_charge_variance_model'][0] = test_results['normal_charge_variance_model'][0] ** 0.5
pd.DataFrame(test_results, index=['MSE', 'Mean absolute error']).T

In [ ]:
variance_model_normal_charge.summary()

# Transfer Learning

### Model Building
Freeze last layer for TF

In [ ]:
variance_model_fast_charge = variance_model_normal_charge

In [ ]:
for layer in variance_model_fast_charge.layers[:-1]:
  layer.trainable = False

### Fast-charge Data Train-Test split

In [ ]:
fast_charge_train_ds = fast_charge_dataset.iloc[0::2, :]
fast_charge_test_ds = fast_charge_dataset.iloc[1::2, :]
sns.pairplot(fast_charge_train_ds[['cycle_life', 'QDiffLinVar']], diag_kind='kde')
sns.pairplot(fast_charge_test_ds[['cycle_life', 'QDiffLinVar']], diag_kind='kde')
# fast_charge_test_ds

In [ ]:
fastcharge_train_features = fast_charge_train_ds.copy()
fastcharge_test_features = fast_charge_test_ds.copy()

fast_train_labels = fastcharge_train_features.pop('cycle_life')
fast_test_labels = fastcharge_test_features.pop('cycle_life')


### Compile and fit

In [ ]:
variance_model_fast_charge2 = tf.keras.Sequential([
    QDiffLinVar_normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
variance_model_fast_charge2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
    )
variance_model_fast_charge.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
    )

In [ ]:
%%time
history = variance_model_fast_charge.fit(
    fastcharge_train_features['QDiffLinVar'],
    fast_train_labels,
    epochs=1000,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
%%time
history2 = variance_model_fast_charge2.fit(
    fastcharge_train_features['QDiffLinVar'],
    fast_train_labels,
    epochs=1000,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

### Plot loss graph

In [ ]:
def plot_loss(history):
  plt.plot(np.sqrt(history.history['loss']), label='loss')
  plt.plot(np.sqrt(history.history['val_loss']), label='val_loss')
  # plt.ylim([200, 130])
  plt.ylim([50, 300])
  plt.xlabel('Epoch')
  plt.ylabel('Error [cycles]')
  plt.legend()
  plt.grid(True)

plot_loss(history)
plot_loss(history2)

In [ ]:
hist = pd.DataFrame(history.history)
hist = hist.pow(0.5) # Power 1/2 is the same as square root
hist['epoch'] = history.epoch
hist



In [ ]:
hist2 = pd.DataFrame(history2.history)
hist2 = hist.pow(0.5)
hist2['epoch'] = history2.epoch
hist2

In [ ]:
test_results['fast_charge_variance_model'] = variance_model_fast_charge.evaluate(
    fastcharge_test_features['QDiffLinVar'],
    fast_test_labels, verbose=1) #sqrt for mse
test_results['fast_charge_variance_model2'] = variance_model_fast_charge2.evaluate(
    fastcharge_test_features['QDiffLinVar'],
    fast_test_labels, verbose=1) #sqrt for mse

In [ ]:
def plot_prediction(y_train, y_test):
  plt.axes(aspect='equal')
  plt.scatter(y_train, fast_train_labels, label='Predictions (train)')
  plt.scatter(y_test, fast_test_labels, label='Predictions (test)')
  lims = [0, 2000]
  plt.xlim(lims)
  plt.ylim(lims)
  plt.plot(lims, lims, 'k', )
  plt.xlabel('Predicted Cycle life')
  plt.ylabel('Actual Cycle life')
  plt.legend()

fast_train_prediction = variance_model_fast_charge.predict(fast_charge_train_ds['QDiffLinVar'])
fast_test_prediction = variance_model_fast_charge.predict(fast_charge_test_ds['QDiffLinVar'])
plot_prediction(fast_train_prediction, fast_test_prediction)



In [ ]:
fast_train_prediction2 = variance_model_fast_charge2.predict(fast_charge_train_ds['QDiffLinVar'])
fast_test_prediction2 = variance_model_fast_charge2.predict(fast_charge_test_ds['QDiffLinVar'])
plot_prediction(fast_train_prediction2, fast_test_prediction2)

In [ ]:
test_results['fast_charge_variance_model'][0] = test_results['fast_charge_variance_model'][0] ** 0.5
test_results['fast_charge_variance_model2'][0] = test_results['fast_charge_variance_model2'][0] ** 0.5
pd.DataFrame(test_results, index=['MSE', 'Mean absolute error']).T

# Scratch Test

In [ ]:
# model = tf.keras.Sequential(
#     [
#         layers.Dense(2, activation="relu"),
#         layers.Dense(3, activation="relu"),
#         layers.Dense(4),
#     ]
# ) 
# model.summary()